In [1]:
from rnn_scratch import load_data,grad_clipping,train_and_predict
import sys
sys.path.append('..')
import util


In [2]:
content,idx_to_char,char_to_idx,idx_corpus,vocab_size = load_data()

64925


In [3]:
ctx = util.try_gpu()

In [4]:
import mxnet.ndarray as nd
import mxnet as mx
def get_params(input_dim,hidden_dim,output_dim,ctx=mx.cpu()):
    # reset gate, update date
    std = .01
    W_xr = nd.random_normal(scale = std,shape=(input_dim,hidden_dim),ctx=ctx)
    W_hr = nd.random_normal(scale =std,shape=(hidden_dim,hidden_dim),ctx=ctx)
    b_r = nd.zeros(hidden_dim,ctx=ctx)
    #update gate
    W_xz = nd.random_normal(scale = std,shape=(input_dim,hidden_dim),ctx=ctx)
    W_hz = nd.random_normal(scale = std,shape=(hidden_dim,hidden_dim),ctx=ctx)
    b_z = nd.zeros(hidden_dim,ctx=ctx)
    
    #candidate hidden state
    W_xh = nd.random_normal(scale = std,shape=(input_dim,hidden_dim),ctx=ctx)
    W_hh = nd.random_normal(scale = std,shape=(hidden_dim,hidden_dim),ctx=ctx)
    b_h = nd.zeros(hidden_dim,ctx=ctx)
    
    #output gate
    W_hy = nd.random_normal(scale = std,shape=(hidden_dim,output_dim),ctx=ctx)
    b_y = nd.zeros(output_dim,ctx=ctx)
    
    params = [W_xr,W_hr,b_r,W_xz,W_hz,b_z,W_xh,W_hh,b_h,W_hy,b_y]
    for param in params:
        param.attach_grad()
    return params

def gated_rnn(inputs,states,*params):
    # R = sigmoid(XW_xr + H*W_hr +  b_r), output dim: n*hidden_dim
    # Z = sigmoid(X*W_xz + H*W_hz + b_z), output dim: n*hidden_dim
    # H_tilde = tanh(X*W_xh + R\codt H *W_hh+b_h) ,output dim: n*hidden_dim
    # H = Z \cdot H + （1-Z) \cdot H_tilde, output dim: n*hidden_dim
    W_xr,W_hr,b_r,W_xz,W_hz,b_z,W_xh,W_hh,b_h,W_hy,b_y = params
    outputs = []
    H = states
    for x in inputs:
        R = nd.sigmoid(nd.dot(x,W_xr)+nd.dot(H,W_hr)+b_r)
        Z = nd.sigmoid(nd.dot(x,W_xz)+nd.dot(H,W_hz)+b_z)
        H_tilde = nd.tanh(nd.dot(x,W_xh)+ nd.multiply(R,nd.dot(H,W_hh)) + b_h)
        H = nd.multiply(Z,H)+ nd.multiply(1-Z,H_tilde)
        # output
        #out = nd.tanh(nd.dot(H,W_hy)+b_y)
        out =nd.dot(H,W_hy)+b_y
        outputs.append(out)
    return  outputs,H


    
    

In [5]:
# setup parameters
batch_size = 32
num_steps = 35
input_dim = vocab_size
output_dim = vocab_size
hidden_dim = 256
learning_rate = 0.2
num_epochs = 20
train_and_predict(gated_rnn,get_params=get_params,num_steps = num_steps,idx_corpus = idx_corpus,num_epochs = num_epochs,batch_size=batch_size,
                 vocab_size = vocab_size,hidden_dim = hidden_dim,learning_rate = learning_rate,ctx= ctx)


Epoch：0  Prelexity: 693.012503
Epoch：1  Prelexity: 454.839552
Epoch：2  Prelexity: 416.882003
Epoch：3  Prelexity: 393.302853
Epoch：4  Prelexity: 382.767116
Epoch：5  Prelexity: 372.594198
Epoch：6  Prelexity: 366.150443
Epoch：7  Prelexity: 360.103871
Epoch：8  Prelexity: 350.509244
Epoch：9  Prelexity: 342.276332
Epoch：10  Prelexity: 336.862112
Epoch：11  Prelexity: 331.177544
Epoch：12  Prelexity: 326.701156
Epoch：13  Prelexity: 317.123575
Epoch：14  Prelexity: 312.108437
Epoch：15  Prelexity: 303.806393
Epoch：16  Prelexity: 295.238410
Epoch：17  Prelexity: 284.992714
Epoch：18  Prelexity: 276.522217
Epoch：19  Prelexity: 268.442386


In [6]:
vocab_size

1465